# Cryptocurrency
#### Group Member: Weiqiu Liu, Ming Fang 

In [1]:
import pandas

## read files

In [2]:
inputs = pandas.read_csv('inputs.csv', header=None)
outputs = pandas.read_csv('outputs.csv', header=None)
tags = pandas.read_csv('tags.csv')
transactions = pandas.read_csv('transactions.csv', header=None)
inputs.columns = ["id", "tx_id", "sig_id", "output_id"]
outputs.columns = ["id", "tx_id", "pk_id", "value"]
tags.columns = ["type", "service", "pk_id"]
transactions.columns = ['id', "block_id"]

## 1 Basic statistic
### 1.1 Transactions

In [3]:
def answer_1_1():
    print ("------------------- 1.1 -------------------")
    print ("How many transactions were there in total?")
    print (len(transactions['id'].unique()))
    print ("Of these, how many transactions had one input and two outputs?")
    count_outputs = outputs['tx_id'].value_counts()
    count_inputs = inputs['tx_id'].value_counts()
    count_outputs_1 = count_outputs[count_outputs == 1]
    count_outputs_2 = count_outputs[count_outputs == 2]
    count_inputs_1 = count_inputs[count_inputs == 1]
    print (len(count_outputs_2[count_outputs_2.index.isin(count_inputs_1.index)]))
    print ("How many transactions had one input and one output?")
    print (len(count_outputs_1[count_outputs_1.index.isin(count_inputs_1.index)]))

In [4]:
answer_1_1()

------------------- 1.1 -------------------
How many transactions were there in total?
216626
Of these, how many transactions had one input and two outputs?
44898
How many transactions had one input and one output?
160780


### 1.2 UTXOs

In [147]:
def answer_1_2():
    print ("------------------- 1.2 -------------------")
    temp_outputs = outputs[['id','pk_id','value']]
    temp_inputs = inputs['output_id']
    UTXOs = temp_outputs[~temp_outputs['id'].isin(temp_inputs)].set_index('id')
    print ("How many UTXOs exist, as of the last block of the dataset?")
    print (len(UTXOs))
    print ("Which UTXO has the highest associated value?")
    print ("output id:", UTXOs.value.idxmax())
    print ("value:", UTXOs.value.max())
    return UTXOs

In [148]:
UTXOs = answer_1_2()

------------------- 1.2 -------------------
How many UTXOs exist, as of the last block of the dataset?
71906
Which UTXO has the highest associated value?
output id: 170430
value: 9000000000000


### 1.3 Public keys

In [7]:
def answer_1_3():
    print ("------------------- 1.3 -------------------")
    print ("How many distinct public keys were used across all blocks in the dataset?")
    #pk_ids = pandas.concat([outputs['pk_id'], tags['pk_id']], axis=0, ignore_index=True).unique()
    print (len(outputs['pk_id'].unique()))
    print ("Which public key received the highest number of bitcoins, and how many bitcoins has it received?")
    temp = outputs
    temp = temp.groupby('pk_id')['value'].sum()
    print("pk_id:", temp.idxmax(), "value:", temp.max())
    print ("Which public key acted as an output the most number of times, and how many times did it act as output?")
    count_pk = outputs['pk_id'].value_counts()
    print ("pk_id:", count_pk.idxmax(), "value:", count_pk.max())

In [418]:
answer_1_3()
# outputs.loc[outputs['pk_id']==148105]

------------------- 1.3 -------------------
How many distinct public keys were used across all blocks in the dataset?
174702
Which public key received the highest number of bitcoins, and how many bitcoins has it received?
pk_id: 148105 value: 27375023000000
Which public key acted as an output the most number of times, and how many times did it act as output?
pk_id: 148105 value: 5498


### 1.4 Invalid transactions

In [9]:
def answer_1_4():
    print ("------------------- 1.4 -------------------")
    ## double spend
    temp_spend = inputs
    temp_spend = temp_spend.loc[(temp_spend['output_id'].duplicated()) & (temp_spend['output_id'] != -1)]
    i = 0
    for _, row in temp_spend.iterrows():
        i += 1
        print (str(i)+". double spend:")
        print ("id:", row.id, "tx_id:", row.tx_id, "sig_id:", row.sig_id, "output_id:", row.output_id)
        print ("spent in:")
        s = inputs.loc[(inputs['output_id'] == row.output_id) & (inputs['id'] != row.id)]
        #print ("id:", s.id, "tx_id:", s.tx_id, "sig_id:", s.sig_id, "output_id:", s.output_id)
        row = s.reset_index().loc[0]
        print ("id:", row.id, "tx_id:", row.tx_id, "sig_id:", row.sig_id, "output_id:", row.output_id)
        print ("")
        
    ## ouput > input
    temp_input = inputs
    temp_output = outputs
    money_in = pandas.merge(temp_input[['tx_id','output_id']],temp_output[['id','value']],left_on='output_id',right_on='id')[['tx_id','value']]
    money_in = money_in.groupby('tx_id')['value'].sum()
    dict_money_in = {'tx_id':money_in.index,'money_in':money_in.values}
    money_in = pandas.DataFrame(dict_money_in)
    money_out = temp_output.groupby('tx_id')['value'].sum()
    dict_money_out = {'tx_id':money_out.index,'money_out':money_out.values}
    money_out = pandas.DataFrame(dict_money_out)
    transactions = pandas.merge(money_in[['tx_id','money_in']], money_out[['tx_id','money_out']], on='tx_id')
    invalid_tx = transactions.loc[(transactions['money_in']<transactions['money_out'])]
    for _, row in invalid_tx.iterrows():
        i += 1
        print (str(i)+". output > input:")
        print ("tx_id:", row.tx_id, "money_in:", row.money_in, "money_out:", row.money_out)
        print ("money_in:")
        j = 0
        for _, row1 in inputs.loc[inputs['tx_id']==row.tx_id].iterrows():
            for _, row2 in outputs.loc[outputs['id']==row1.output_id].iterrows():
                j += 1
                print (str(j)+") input_id:", row1.tx_id, "UTXO_id:", row1.output_id, "owner_id:", row1.sig_id, "value:",row2.value)
        print ("money_out:")
        j = 0
        for _, row1 in outputs.loc[outputs['tx_id']==row.tx_id].iterrows():
            j += 1
            print (str(j)+") output_id:", row1.id, "payee_id:", row1.pk_id, "value:", row1.value)

In [10]:
answer_1_4()

------------------- 1.4 -------------------
1. double spend:
id: 12820 tx_id: 12152 sig_id: 7941 output_id: 7998
spent in:
id: 8666 tx_id: 8231 sig_id: 7941 output_id: 7998

2. double spend:
id: 33114 tx_id: 30446 sig_id: 21807 output_id: 21928
spent in:
id: 33113 tx_id: 30446 sig_id: 21807 output_id: 21928

3. double spend:
id: 76750 tx_id: 61845 sig_id: 138980 output_id: 65403
spent in:
id: 76747 tx_id: 61843 sig_id: 138980 output_id: 65403

4. double spend:
id: 279609 tx_id: 207365 sig_id: 163625 output_id: 249860
spent in:
id: 275614 tx_id: 204751 sig_id: 163625 output_id: 249860

5. output > input:
tx_id: 100929 money_in: 5000000000 money_out: 5000000010
money_in:
1) input_id: 100929 UTXO_id: 37553 owner_id: 37334 value: 5000000000
money_out:
1) output_id: 118119 payee_id: 60205 value: 2500000000
2) output_id: 118120 payee_id: 142642 value: 2500000010


## 2 Clustering
### generate cluster dataframe

In [354]:
# def generate_clusters():
#     # get unique dataframe with columns = ['tx_id','sig_id']
#     clusters = inputs.drop_duplicates(['tx_id','sig_id'])[['tx_id','sig_id','output_id']]
#     # set all sig_id to have default cluster_id = 0
#     clusters['cluster_id'] = 0
#     # init max_cluster_id = 0, max_cluster_id += 1 when assigned
#     max_cluster_id = 0
#     # init a dict to store tx_id ever appeared, key: tx_id, value: cluster_id
#     appeared_tx_id_dict = {}
#     # init a dict to store sig_id ever appeared, key: sig_id, value: cluster_id
#     appeared_sig_id_dict = {}
#     # traverse all rows
#     # 'appeared' means: it has already been assinged a cluster_id
#     # pk_id == sig_id
#     for _, row in clusters.iterrows():
#         if row.tx_id not in appeared_tx_id_dict.keys():
#             if row.sig_id in appeared_sig_id_dict.keys():
#                 # if signature id appeared but not transaction id,
#                 # we add {tx_id: cluster_id} to dict and set cluster_id of current row
#                 appeared_tx_id_dict[row.tx_id] = appeared_sig_id_dict[row.sig_id]
#                 row.cluster_id = appeared_sig_id_dict[row.sig_id]
#             else:
#                 # if neither appeared,
#                 # we need to check whether there were other signature appeared, in the same transaction
#                 for _, row1 in clusters.loc[clusters['tx_id']==row.tx_id].iterrows():
#                     if row1.sig_id in appeared_sig_id_dict.keys():
#                         # if other signature S appeared in the same transaction,
#                         # then we set cluster_id of current row to S's cluster_id
#                         # and add {tx_id: cluster_id}, {sig_id: cluster_id} to dict respectively
#                         appeared_tx_id_dict[row.tx_id] = appeared_sig_id_dict[row1.sig_id]
#                         appeared_sig_id_dict[row.sig_id] = appeared_sig_id_dict[row1.sig_id]
#                         row.cluster_id = appeared_sig_id_dict[row1.sig_id]
#                         break
#                 if row.cluster_id == 0:
#                     # if all signature id in the same transaction not appeared before,
#                     # we assign to a new cluster_id and update the dict
#                     max_cluster_id += 1
#                     appeared_tx_id_dict[row.tx_id] = max_cluster_id
#                     appeared_sig_id_dict[row.sig_id] = max_cluster_id
#                     row.cluster_id = max_cluster_id
#         else:
#             # if tx_id appeared before, set the row cluster_id to corresponding value (from the dict)
#             # and update the dict
#             row.cluster_id = appeared_tx_id_dict[row.tx_id]
#             if row.sig_id in appeared_sig_id_dict.keys():
#                 pass
#             else:
#                 appeared_sig_id_dict[row.sig_id] = appeared_tx_id_dict[row.tx_id]
#     return clusters


def generate_clusters():
    max_cluster_id = 0
    clusters = pandas.DataFrame()
    clusters['pk_id'] = inputs['sig_id'].unique()
    clusters = clusters.set_index('pk_id')
    clusters['cluster_id'] = 0

    def get_all_related_pk_id(pk_id,pk_id_list):
        tx_id_array = inputs.loc[inputs['sig_id']==pk_id,'tx_id'].unique()
        if clusters.loc[pk_id].cluster_id != 0:
            print(123456)
        for tx_id_related in tx_id_array:
            pk_id_array = inputs.loc[inputs['tx_id']==tx_id_related,'sig_id'].unique()
            for pk_id_related in pk_id_array:
                if pk_id_related not in pk_id_list:
                    pk_id_list += [pk_id_related]
                    if pk_id_related == pk_id:
                        continue
                    pk_id_list = get_all_related_pk_id(pk_id_related,pk_id_list)
        return pk_id_list

    for pk_id in clusters.index:
        if clusters.loc[pk_id].cluster_id != 0:
            continue
        pk_id_list = []
        pk_id_list = get_all_related_pk_id(pk_id,pk_id_list)
        max_cluster_id += 1
        clusters.loc[pk_id_list] = max_cluster_id
    return clusters

In [365]:
#import time
#t1 = time.clock()
clusters = generate_clusters()
#print (time.clock()-t1)
#clusters.columns = ['tx_id','pk_id','output_id','cluster_id']

In [415]:
len(clusters)

110316

### 2.1 Specific cluster

In [363]:
def answer_2_1():
    print ("------------------- 2.1 -------------------")
    print ("How big is the cluster that contains public key (pk_id) 41442?")
    cluster_id_41442 = clusters.loc[41442].cluster_id
    pk_id_in_cluster = clusters.loc[clusters['cluster_id']==cluster_id_41442].index
    print (clusters['cluster_id'].value_counts()[cluster_id_41442])
    print ("Identify the cluster according to its keys with the lowest and highest numeric values.")
    print ("lowest pk_id:", pk_id_in_cluster.min())
    print ("highest pk_id:", pk_id_in_cluster.max())

In [364]:
answer_2_1()

------------------- 2.1 -------------------
How big is the cluster that contains public key (pk_id) 41442?
50
Identify the cluster according to its keys with the lowest and highest numeric values.
lowest pk_id: 40284
highest pk_id: 41911


### 2.2 Biggest cluster

In [370]:
def answer_2_2():
    print ("------------------- 2.2 -------------------")
    print ("Which cluster has the largest number of keys, and how many keys does it contain?")
    temp = clusters
    cluster_id, number_of_keys = temp['cluster_id'].value_counts().idxmax(), temp['cluster_id'].value_counts().max()
    print ("number of keys:",number_of_keys)
    print ("Identify it according to its keys with the lowest and highest numeric values.")
    print ("lowest pk_id:",temp.loc[temp['cluster_id']==cluster_id].index.min())
    print ("highest pk_id:",temp.loc[temp['cluster_id']==cluster_id].index.max())

In [371]:
answer_2_2()

------------------- 2.2 -------------------
Which cluster has the largest number of keys, and how many keys does it contain?
number of keys: 921
Identify it according to its keys with the lowest and highest numeric values.
lowest pk_id: 29823
highest pk_id: 173091


### 2.3 Richest cluster

In [428]:
def answer_2_3():
    print ("------------------- 2.3 -------------------")
    print ("As of the last block in the dataset, which cluster controls the most unspent bitcoins, and how many bitcoins does it control? Again, identify it according to its keys with the lowest and highest numeric values.")
    pk_id_grouped_UTXOs_dict = UTXOs.groupby('pk_id')['value'].sum()
    pk_id_grouped_UTXOs = pandas.DataFrame({'pk_id':pk_id_grouped_UTXOs_dict.index,'value':pk_id_grouped_UTXOs_dict.values}).set_index('pk_id')
    clusters_UTXOs = pandas.merge(clusters,pk_id_grouped_UTXOs,left_index=True,right_index=True,how='inner')
    clusters_UTXOs = clusters_UTXOs.groupby('cluster_id')['value'].sum()
    print ("highest unspent bitcoin:",clusters_UTXOs.max())
    cluster_id = clusters_UTXOs.idxmax()
    print ("lowest pk_id:",clusters.loc[clusters['cluster_id']==cluster_id].index.min())
    print ("highest pk_id:",clusters.loc[clusters['cluster_id']==cluster_id].index.max())
    print ("Which transaction is responsible for sending the largest number of bitcoins to this entity (i.e., to one or more of the keys in this cluster)?")
    cluster_with_highest_UTXO = clusters.loc[clusters['cluster_id']==cluster_id].index
    output_to_this_cluster = outputs.loc[outputs['pk_id'].isin(cluster_with_highest_UTXO)]
    tx = output_to_this_cluster.groupby('tx_id').value.sum()
    print ("tx_id:",tx.idxmax(),"value",tx.max())

In [429]:
answer_2_3()


------------------- 2.3 -------------------
As of the last block in the dataset, which cluster controls the most unspent bitcoins, and how many bitcoins does it control? Again, identify it according to its keys with the lowest and highest numeric values.
highest unspent bitcoin: 4755624000000
lowest pk_id: 37214
highest pk_id: 39508
Which transaction is responsible for sending the largest number of bitcoins to this entity (i.e., to one or more of the keys in this cluster)?
tx_id: 38632 value 1186780000000


### 2.4 Heuristics

In [423]:
outputs.loc[outputs['tx_id']==38632]

,id,tx_id,pk_id,value
38728,38729,38632,38497,3220000000
38729,38730,38632,38498,1186780000000


In [431]:
clusters.to_csv('clusters.csv')

In [436]:
# t = clusters.loc[clusters['cluster_id']!=1]
# cluster_id_df = clusters['cluster_id'].unique()
# for cid in cluster_id_df:
#     tx_id_df = clusters.loc[clusters['cluster_id']==cid,'tx_id'].unique()
#     for tid in tx_id_df:
#         oids = outputs.loc[outputs['tx_id']==tid,'pk_id'].unique()
#         flag = 1
#         for oid in oids:
#             if cid not in(clusters.loc[clusters['pk_id']==oid,'cluster_id'].unique()):
#                 flag = 0
#                 break
#         if flag:
#             print (cid, tid)
        
clusters_old = pandas.read_csv('clusters_old.csv')
clusters_old = clusters_old[['pk_id','cluster_id']]
vc = clusters_old['pk_id'].value_counts()
for pk_id in vc.index:
    cid = 0
    if vc[pk_id] != 1:
        for _,row in clusters_old.loc[clusters_old['pk_id']==pk_id].iterrows():
            if cid != 0 and cid != row.cluster_id:
                print (pk_id,cid,row.cluster_id)
            cid = row.cluster_id

145453 49856 48223
145453 48223 49856
145453 49856 48223
145453 48223 49856
145453 49856 48223
145453 48223 49856
145453 49856 48223
145453 48223 49856
145453 49856 48223
145453 48223 49856
145453 49856 48223
145453 48223 49856
145453 49856 48223
145453 48223 49856
146501 48223 49856
146501 49856 48223
146501 48223 49856
146501 49856 48223
146501 48223 49856
146501 49856 48223
146501 48223 49856
146501 49856 48223
146501 48223 49856
146501 49856 48223
146501 48223 49856
146501 49856 48223
146501 48223 49856
146501 49856 48223
152119 49982 50149
151910 49873 48223
151910 48223 49873
146386 48224 48223
146386 48223 48224
105710 26247 15727
105710 15727 4738
105710 4738 26247
105710 26247 6501
105710 6501 26247
105710 26247 4738
105710 4738 26247
105710 26247 4738
105710 4738 26247
105710 26247 49943
105710 49943 26247
105710 26247 51835
105710 51835 26247
105710 26247 51835
105710 51835 26247
105710 26247 51835
105710 51835 26247
105710 26247 51835
105710 51835 26247
105710 26247 58034
1

95581 17111 4563
95581 4563 17333
95581 17333 17111
95581 17111 21450
95581 21450 17111
68707 5895 9306
68707 9306 5895
113040 29348 29314
113040 29314 29348
83344 4563 11693
83344 11693 4563
83344 4563 17333
83344 17333 4563
83344 4563 4569
106023 21961 17308
106023 17308 16366
55540 4048 924
94577 21581 4563
94577 4563 21450
94577 21450 21581
94577 21581 29032
94577 29032 29325
94577 29325 18603
93182 21001 15727
93182 15727 16534
93182 16534 15727
75566 23514 25957
75566 25957 23514
75566 23514 25957
102220 24839 24943
102220 24943 24756
102220 24756 24839
114016 32901 28251
114016 28251 32901
50872 818 4294
50872 4294 235
50872 235 802
50872 802 818
102234 24841 25005
102234 25005 24841
78026 11693 4563
78026 4563 11693
83507 15727 16534
83507 16534 15727
109662 26468 27016
109662 27016 27498
109662 27498 26468
87643 4738 26247
87643 26247 4738
87643 4738 9942
87643 9942 31342
87643 31342 4738
89302 17333 17111
89302 17111 17333
89302 17333 4563
89302 4563 17333
108609 27416 4738
1

115342 30535 30513
115342 30513 30088
77139 34075 34074
115393 30513 30117
121267 33625 33607
121267 33607 33625
168468 58247 56372
115714 30905 30712
115714 30712 30905
115120 30348 30670
114844 30203 30081
114844 30081 30183
162230 54531 50830
162230 50830 54531
114340 29911 29866
114340 29866 29911
131463 39488 33884
115290 30579 30459
115290 30459 30513
94294 15727 26247
94294 26247 6501
115905 30883 30839
115905 30839 30774
115006 30292 30361
115006 30361 30292
114876 30203 29870
114876 29870 30203
115054 30356 30355
115071 30349 30350
115071 30350 30463
62377 6114 6392
115901 31269 30773
132563 36275 37004
132563 37004 36509
114955 30262 30091
114955 30091 30262
115649 30656 30654
132406 36275 37004
132406 37004 36509
114624 30097 30105
114624 30105 30097
114224 29837 29836
114224 29836 29774
115677 30670 30558
111483 30284 24756
111483 24756 30284
114752 30111 30081
114752 30081 30111
115338 30516 30593
115338 30593 30513
114881 30213 30081
114881 30081 30213
77459 34074 34085
7

118128 31822 31808
117387 31514 31470
20828 117 57
121197 33601 29622
96323 23272 9714
127595 36275 36509
114282 29864 29866
118084 31914 31761
126874 36275 36509
132296 36275 37004
118291 31914 31946
118444 31998 31881
93349 21485 7361
131987 36275 37004
126893 36275 36509
118257 31966 31998
127505 36275 36509
115885 30781 30797
118294 31913 31686
116913 31253 31514
124058 34951 37004
108615 27748 25957
115795 30733 30712
170609 26247 56040
75734 9635 9634
117516 31538 31514
117425 31506 31300
161448 51835 26247
117520 31539 31514
121167 33598 33585
115781 30777 31269
131530 36275 37004
117575 31561 31514
127252 36275 36509
119113 32281 31996
128294 36509 37004
117662 31941 31592
118842 32141 32284
117599 31586 31761
137127 41059 41207
157163 52415 51263
117471 31516 31514
128821 37004 36509
118192 31863 31686
117363 31466 30896
120061 35037 8225
132134 36275 36509
159451 39418 36505
11404 89 134
105045 25962 37004
128835 37004 36509
127508 36275 36509
114404 30204 29891
117087 31398 

In [ ]:
cluster_array = clusters['cluster_id'].unique()

for cluster_id in cluster_array:
    pk_id_array = clusters.loc[clusters['cluster_id']==cluster_id].index
    if len(pk_id_array) > 1:
        output_id_array = outputs.loc[outputs['pk_id'].isin(pk_id_array)]
        tx_id_array = output_id_array['tx_id'].unique()
        for tx_id in tx_id_array:
            l1 = len(inputs.loc[inputs['tx_id']==tx_id])
            l2 = len(outputs.loc[outputs['tx_id']==tx_id])
            if l1 == 1:
                continue
            #print (l1,l2)
            if 2*l1 > l2:
                print (tx_id,l1,cluster_id)


558 2 7
559 2 7
560 2 7
4466 5 42
4469 2 42
5282 4 47
6258 2 55
20010 121 57
20920 111 57
21714 36 57
6845 6 61
7261 3 63
10668 7 79
10811 6 79
28702 168 80
22052 6 89
22263 14 89
22564 13 89
23553 8 89
23796 6 89
23921 16 89
24102 13 89
24382 10 89
13169 14 93
14171 2 98
23238 101 122
25641 101 122
24602 4 132
24603 11 132
25005 74 132
25266 81 134
25378 7 134
25793 40 135
25964 7 136
26146 16 139
26257 10 139
26582 42 140
27134 2 143
26996 20 145
26998 20 145
27006 11 145
27138 10 145
27718 77 146
29015 2 153
29215 20 153
29273 10 153
29644 60 153
29646 7 153
29332 10 161
29996 7 161
34975 8 161
28971 18 162
29377 4 165
30092 17 175
34639 101 177
34640 115 177
36030 2 189
36050 2 189
36784 3 189
41246 4 189
35893 6 190
35895 5 190
37568 20 200
45114 17 200
46479 24 200
37584 5 215
37810 2 221
33270 79 233
38632 238 233
41246 4 233
42059 14 233
42719 68 233
44075 11 233
44208 6 233
44239 380 233
44460 83 233
44875 164 233
44964 80 233
45218 14 233
45311 35 233
45409 5 233
45684 3 233


84136 2 976
85399 13 976
87473 2 976
88844 7 976
88923 2 976
89079 3 976
89213 3 976
93153 15 976
97245 7 976
97631 2 976
98039 3 976
98166 3 976
98239 3 976
98550 2 976
98678 4 976
99417 2 976
99948 2 976
101728 2 976
102223 2 976
102296 2 976
103417 3 976
103760 2 976
105471 2 976
106291 4 976
106935 2 976
107200 2 976
107704 2 976
109064 2 976
109416 2 976
111109 2 976
111743 2 976
112410 3 976
114102 2 976
115589 2 976
116861 2 976
125361 2 976
129504 2 976
145661 3 976
55182 100 4015
55996 3 4015
57362 3 4015
57501 2 4015
60972 965 4015
71172 2 4015
56177 6 4077
56494 4 4080
56495 2 4080
56496 2 4080
56723 2 4080
56753 2 4080
57354 4 4080
57575 4 4080
57576 2 4080
58054 2 4080
58516 2 4080
58944 4 4080
59174 3 4080
59367 2 4080
59943 2 4080
60436 6 4080
60677 2 4080
60682 3 4080
60684 8 4080
61015 2 4080
61017 4 4080
61350 2 4080
61864 3 4080
61865 2 4080
62618 3 4080
63126 4 4080
63127 3 4080
63128 4 4080
63510 2 4080
63511 2 4080
64470 3 4080
64471 3 4080
65001 2 4080
65002 2 40

83025 3 4289
84059 2 4289
84606 100 4289
84619 100 4289
84620 100 4289
84622 200 4289
84623 199 4289
84860 3 4289
84890 3 4289
86092 2 4289
86300 3 4289
86440 2 4289
86714 47 4289
86965 55 4289
87434 2 4289
87679 3 4289
88616 4 4289
88619 8 4289
89059 2 4289
89393 3 4289
89531 2 4289
89814 3 4289
89845 2 4289
89849 3 4289
90017 2 4289
90144 3 4289
90760 2 4289
93039 824 4289
93042 11 4289
93285 2 4289
93295 2 4289
93336 5 4289
93393 2 4289
97317 2 4289
97478 2 4289
97483 39 4289
97508 3 4289
97530 2 4289
97554 2 4289
97829 4 4289
98482 2 4289
98485 2 4289
98487 4 4289
98553 3 4289
98648 2 4289
99241 3 4289
100360 36 4289
100560 3 4289
100641 4 4289
100909 4 4289
100998 6 4289
101006 11 4289
101212 3 4289
101656 2 4289
101676 4 4289
102228 3 4289
103125 28 4289
103134 16 4289
103449 5 4289
103612 4 4289
103733 5 4289
104368 2 4289
104371 4 4289
104415 2 4289
104426 9 4289
104551 2 4289
104977 2 4289
105325 2 4289
105545 2 4289
106532 8 4289
107027 22 4289
108453 2 4289
108536 2 4289
108

64881 3 4455
65061 2 4455
65065 4 4455
65143 2 4455
65335 4 4455
65340 2 4455
65343 2 4455
66061 10 4455
66070 11 4455
66071 8 4455
66598 12 4455
66599 3 4455
66600 6 4455
66966 2 4455
66970 7 4455
66973 5 4455
67429 3 4455
67433 11 4455
67434 9 4455
67649 3 4455
67666 9 4455
67934 4 4455
67943 5 4455
68156 3 4455
68160 4 4455
68230 4 4455
68234 53 4455
68625 2 4455
68626 5 4455
68628 8 4455
68836 4 4455
68838 5 4455
69027 5 4455
69029 4 4455
69030 2 4455
69454 7 4455
69459 8 4455
69705 3 4455
69810 4 4455
69817 6 4455
70019 3 4455
70022 3 4455
70254 3 4455
70394 2 4455
70421 2 4455
70599 5 4455
70603 3 4455
70606 2 4455
70714 2 4455
70716 2 4455
70882 5 4455
70939 6 4455
70942 2 4455
70943 2 4455
71182 3 4455
71183 3 4455
71184 3 4455
71185 2 4455
71186 3 4455
71192 4 4455
71654 6 4455
71655 6 4455
71659 7 4455
71661 11 4455
71700 2 4455
71909 4 4455
71910 3 4455
56435 11 4457
58142 7 4457
58996 4 4457
63476 11 4457
87898 4 4457
57534 18 4476
64380 5 4476
87463 20 4476
46649 2 4495
59

92778 2 12625
85719 2 15559
89000 2 15559
86578 2 15621
84107 2 15733
84170 2 15733
82427 3 15760
82449 6 15760
84783 3 15760
86170 2 15760
86171 2 15760
86173 3 15760
86174 3 15760
86175 2 15760
86178 5 15760
87489 2 15760
87490 4 15760
87491 2 15760
87492 2 15760
87493 2 15760
87494 5 15760
87495 2 15760
87497 3 15760
88965 4 15760
88966 3 15760
88977 3 15760
88980 2 15760
88982 5 15760
88984 3 15760
99108 8 15760
122590 3 15760
122591 2 15760
122592 4 15760
122593 7 15760
122595 5 15760
122596 2 15760
122598 3 15760
122599 10 15760
122615 6 15760
87057 2 15910
87066 2 15910
87071 2 15910
87486 2 15910
87814 2 15910
87816 2 15910
87818 2 15910
87819 2 15910
87820 2 15910
87821 2 15910
87831 2 15910
88015 2 15910
102882 12 15910
84878 2 16002
88245 2 16073
88485 2 16073
84863 2 16227
77728 2 16287
80907 4 16287
98775 2 16305
109921 2 16305
84732 2 16306
90766 302 16306
92438 592 16306
92444 3 16306
100974 2 16306
109562 2 16306
109563 2 16306
109588 2 16306
110246 2 16306
110276 2 163

201612 7 21605
202070 4 21605
202453 2 21605
202892 3 21605
203313 5 21605
203808 2 21605
204388 3 21605
204916 3 21605
205346 3 21605
205487 3 21605
205654 2 21605
205804 2 21605
206308 3 21605
207007 3 21605
207631 3 21605
208325 9 21605
208905 7 21605
209620 3 21605
210415 2 21605
211206 4 21605
211903 3 21605
212581 5 21605
213168 2 21605
213718 5 21605
214250 2 21605
215370 4 21605
215990 6 21605
216610 4 21605
98040 2 21614
97325 2 21624
93361 4 21642
97298 4 21642
97322 2 21642
97843 2 21642
98040 2 21642
98082 3 21642
98152 2 21660
112795 2 21660
97360 9 21665
98236 2 21674
100879 8 21674
101007 2 21674
101186 2 21674
125057 8 21674
90017 2 21698
97358 2 21708
97063 2 21801
98296 2 21801
88018 2 21823
77764 2 21829
78116 2 21829
82426 3 21829
88018 2 21829
88020 4 21829
88512 2 21829
93533 2 21829
98657 2 21829
98654 2 21849
98750 3 21853
88445 203 21973
99152 2 22128
69359 3 22179
69361 5 22179
71204 3 22179
88967 10 22179
78139 4 22180
87637 4 22180
87900 5 22180
90095 5 2218

115949 3 23961
92840 5 24001
102337 4 24001
102349 6 24001
102371 4 24001
103256 2 24001
103556 7 24038
104915 5 24038
105305 3 24038
105558 5 24038
105563 3 24038
105565 2 24038
106000 2 24038
106001 2 24038
106004 2 24038
106570 3 24038
106577 5 24038
106581 3 24038
106586 4 24038
107484 2 24038
107486 2 24038
108056 4 24038
108144 2 24038
108152 2 24038
108366 3 24038
109355 3 24038
109380 2 24038
109381 2 24038
109387 2 24038
109390 2 24038
109685 2 24038
110584 2 24038
102676 3 24055
103026 3 24058
102535 4 24071
102543 4 24083
103151 16 24083
103384 4 24083
102529 2 24132
102532 3 24132
103457 2 24200
88304 2 24246
104162 2 24310
104180 2 24318
104181 2 24318
104218 2 24346
103931 2 24389
103958 4 24396
106946 2 24397
85528 2 24456
104219 2 24461
104889 2 24480
104922 2 24480
104924 11 24480
105214 2 24480
105301 9 24480
105304 3 24480
105305 3 24480
105554 2 24480
105563 3 24480
106000 2 24480
106001 2 24480
106560 2 24480
106577 5 24480
106581 3 24480
106586 4 24480
107476 5 24

85781 3 25480
86849 4 25480
106433 2 25504
106778 3 25504
106947 2 25520
106952 2 25526
106958 2 25527
106959 2 25529
106961 2 25530
107042 2 25560
109959 2 25560
110024 2 25560
110028 2 25560
110349 2 25560
113349 2 25560
115302 8 25560
119737 2 25560
134416 3 25560
188176 2 25560
204396 2 25560
106309 3 25586
128566 2 25586
105173 2 25588
106966 4 25602
106967 2 25615
106963 2 25617
107199 2 25618
107201 2 25618
107127 3 25654
100291 2 25666
104779 4 25667
106042 2 25667
99294 8 25688
107455 3 25744
107519 2 25744
107520 2 25744
107521 3 25745
105172 7 25766
92896 2 25810
106479 2 25839
106640 3 25839
106743 4 25839
106745 3 25839
107359 2 25839
108037 8 25839
108136 3 25839
108322 2 25839
108501 3 25839
108550 3 25839
108675 2 25839
108737 2 25839
111156 3 25839
111554 7 25839
112228 2 25839
112452 2 25839
112643 3 25839
113109 2 25839
114149 2 25839
114177 2 25839
114248 3 25839
114298 3 25839
114670 6 25839
114695 4 25839
115262 3 25839
116302 8 25839
117416 2 25839
117953 15 2583

In [444]:
len(inputs.loc[inputs['tx_id']==26131])

1

In [344]:
# max_cluster_id = 0
# clusters_test = pandas.DataFrame()
# clusters_test['pk_id'] = inputs['sig_id'].unique()
# clusters_test = clusters_test.set_index('pk_id')
# clusters_test['cluster_id'] = 0
# for pk_id in clusters_test.index:
#     if clusters_test.loc[pk_id].cluster_id != 0:
#         continue
#     tx_id_array = inputs.loc[inputs['sig_id']==pk_id,'tx_id'].unique()
#     pk_id_list = []
#     for tx_id in tx_id_array:
#         pk_id_array = inputs.loc[inputs['tx_id']==tx_id,'sig_id'].unique()
#         pk_id_list += pk_id_array.tolist()
#         for related_pk_id in pk_id_array:
#             related_cluster_id = clusters_test.loc[related_pk_id].cluster_id
#             if related_cluster_id != 0:
#                 pk_id_list += clusters_test.loc[clusters_test['cluster_id']==related_cluster_id].index.tolist()
#     max_cluster_id += 1
#     pk_id_list = list(set(pk_id_list))
#     clusters_test.loc[pk_id_list] = max_cluster_id



In [345]:
clusters_test

,cluster_id
pk_id,
0,1
10,2
188,3
186,4
368,5
193,5
256,5
294,6
534,7


In [341]:
t = {}
t += clusters_test.loc[clusters_test['cluster_id']==0].index.tolist()
t

TypeError: unsupported operand type(s) for +=: 'dict' and 'list'

In [324]:
clusters_test.loc[clusters_test['cluster_id']==0]

,cluster_id
pk_id,
558,0
1506,0
20380,0
19792,0
19819,0
20125,0
20134,0
19766,0
18635,0


In [325]:
tx_id_array = inputs.loc[inputs['sig_id']==,'tx_id'].unique()
tx_id_array

SyntaxError: invalid syntax (<ipython-input-325-8badaced630e>, line 1)

In [309]:
if (clusters_test.loc[0]==2):
    print(1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [313]:
(clusters_test.loc[0].cluster_id==2)

True